In [1]:
import os
import math
import keras
import tensorflow as tf
from keras_retinanet.models import load_model
import cv2
import time
import numpy as np
from shared import get_session
import matplotlib.pyplot as plt
from paths import *

keras.backend.tensorflow_backend.set_session(get_session())

MODEL_PATH = os.path.join(BASE_DIR, 'snapshots', '8_classes_imgnet_1e5_augmented_70.h5')
model = load_model(MODEL_PATH, backbone_name='resnet50', convert=True, nms=True)

c:\users\mads\desktop\deeple~1\venv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
c:\users\mads\desktop\deeple~1\venv\lib\site-packages\keras\models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [25]:
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box
from keras_retinanet.utils.colors import label_color
from keras_retinanet.layers.filter_detections import filter_detections
from PIL import Image

#The images are not in the github repository
IMAGES_PATH = os.path.join(BASE_DIR, 'Test2')
IMAGES_PATH_ANNOTATED = os.path.join(BASE_DIR, 'Test2_Converted')
images = [f for f in os.listdir(IMAGES_PATH) if os.path.isfile(os.path.join(IMAGES_PATH, f))]
images_amount = len(images)

#Modified draw_caption from https://github.com/fizyr/keras-retinanet/blob/master/keras_retinanet/utils/visualization.py
#Since the text size was hardcoded and we needed to change it.
def draw_caption(image, box, caption):
    """ Draws a caption above the box in an image.
    # Arguments
        image   : The image to draw on.
        box     : A list of 4 elements (x1, y1, x2, y2).
        caption : String containing the text to draw.
    """
    b = np.array(box).astype(int)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    #cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)

#Iterate images
#Inference and drawing of bounding boxes partly inspired by https://github.com/fizyr/keras-retinanet/blob/master/examples/ResNet50RetinaNet.ipynb
for i in range(images_amount):
    start = time.time()
    image = read_image_bgr(os.path.join(IMAGES_PATH, images[i]))
    
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    
    # correct for image scale
    boxes /= scale
    
    #Non-maximum-suppression, since the one in retinanet does not do a good enough job
    #Or we are doing it wrong, idk.
    nms_boxes = tf.image.non_max_suppression(
        boxes[0],
        scores[0],
        30,
        iou_threshold=0.05,
        name=None
    )
    nms_boxes = keras.backend.tensorflow_backend.get_session().run(nms_boxes)
    
    #Iterate suppressed boxes
    for idx in nms_boxes:
        label = labels[0, idx]
        score = scores[0, idx]
        box = boxes[0, idx]

        caption = "{} score: {}".format(label, score)

        if score < 0.2:
            break

        color = label_color(label)
        b = box.astype(int)
        draw_box(draw, b, color=color)
        caption = "{} {:.2f}".format(label, score)
        draw_caption(draw, b, caption)

    
    im = Image.fromarray(draw)
    im.save(os.path.join(IMAGES_PATH_ANNOTATED, images[i]))
    im.close()
    print("Image {} of {} processing time: {}".format(images[i], images_amount, (time.time() - start)))

Image scene00001.jpeg of 2700 processing time: 0.7279708385467529
Image scene00002.jpeg of 2700 processing time: 0.7050309181213379
Image scene00003.jpeg of 2700 processing time: 0.7109947204589844
Image scene00004.jpeg of 2700 processing time: 0.7049756050109863
Image scene00005.jpeg of 2700 processing time: 0.7269988059997559
Image scene00006.jpeg of 2700 processing time: 0.7160317897796631
Image scene00007.jpeg of 2700 processing time: 0.7080280780792236
Image scene00008.jpeg of 2700 processing time: 0.6990013122558594
Image scene00009.jpeg of 2700 processing time: 0.7120020389556885
Image scene00010.jpeg of 2700 processing time: 0.7039685249328613
Image scene00011.jpeg of 2700 processing time: 0.8730316162109375
Image scene00012.jpeg of 2700 processing time: 0.7319695949554443
Image scene00013.jpeg of 2700 processing time: 0.8259990215301514
Image scene00014.jpeg of 2700 processing time: 0.7150261402130127
Image scene00015.jpeg of 2700 processing time: 0.7169725894927979
Image scen

Image scene00126.jpeg of 2700 processing time: 0.7169971466064453
Image scene00127.jpeg of 2700 processing time: 0.7009983062744141
Image scene00128.jpeg of 2700 processing time: 0.7140045166015625
Image scene00129.jpeg of 2700 processing time: 0.7150259017944336
Image scene00130.jpeg of 2700 processing time: 0.7089719772338867
Image scene00131.jpeg of 2700 processing time: 0.7110283374786377
Image scene00132.jpeg of 2700 processing time: 0.7059829235076904
Image scene00133.jpeg of 2700 processing time: 0.718986988067627
Image scene00134.jpeg of 2700 processing time: 0.7070004940032959
Image scene00135.jpeg of 2700 processing time: 0.7110006809234619
Image scene00136.jpeg of 2700 processing time: 0.7180280685424805
Image scene00137.jpeg of 2700 processing time: 0.7190005779266357
Image scene00138.jpeg of 2700 processing time: 0.7421174049377441
Image scene00139.jpeg of 2700 processing time: 1.4665086269378662
Image scene00140.jpeg of 2700 processing time: 0.7090005874633789
Image scene

Image scene00251.jpeg of 2700 processing time: 0.7349977493286133
Image scene00252.jpeg of 2700 processing time: 0.7260024547576904
Image scene00253.jpeg of 2700 processing time: 0.7409965991973877
Image scene00254.jpeg of 2700 processing time: 0.7350287437438965
Image scene00255.jpeg of 2700 processing time: 0.7239995002746582
Image scene00256.jpeg of 2700 processing time: 0.7459971904754639
Image scene00257.jpeg of 2700 processing time: 0.7300000190734863
Image scene00258.jpeg of 2700 processing time: 0.737030029296875
Image scene00259.jpeg of 2700 processing time: 0.7269737720489502
Image scene00260.jpeg of 2700 processing time: 0.74202561378479
Image scene00261.jpeg of 2700 processing time: 0.7259728908538818
Image scene00262.jpeg of 2700 processing time: 0.7380354404449463
Image scene00263.jpeg of 2700 processing time: 0.7279961109161377
Image scene00264.jpeg of 2700 processing time: 0.7350003719329834
Image scene00265.jpeg of 2700 processing time: 0.7250032424926758
Image scene00

Image scene00376.jpeg of 2700 processing time: 0.7980020046234131
Image scene00377.jpeg of 2700 processing time: 0.7579958438873291
Image scene00378.jpeg of 2700 processing time: 0.7490003108978271
Image scene00379.jpeg of 2700 processing time: 0.7670001983642578
Image scene00380.jpeg of 2700 processing time: 0.7620007991790771
Image scene00381.jpeg of 2700 processing time: 0.9699978828430176
Image scene00382.jpeg of 2700 processing time: 0.8179976940155029
Image scene00383.jpeg of 2700 processing time: 0.8480014801025391
Image scene00384.jpeg of 2700 processing time: 0.833000659942627
Image scene00385.jpeg of 2700 processing time: 0.8459987640380859
Image scene00386.jpeg of 2700 processing time: 0.771003246307373
Image scene00387.jpeg of 2700 processing time: 0.8030238151550293
Image scene00388.jpeg of 2700 processing time: 0.7940006256103516
Image scene00389.jpeg of 2700 processing time: 0.8009970188140869
Image scene00390.jpeg of 2700 processing time: 0.927004337310791
Image scene00

Image scene00501.jpeg of 2700 processing time: 0.8189980983734131
Image scene00502.jpeg of 2700 processing time: 0.8019993305206299
Image scene00503.jpeg of 2700 processing time: 0.9050030708312988
Image scene00504.jpeg of 2700 processing time: 0.9069960117340088
Image scene00505.jpeg of 2700 processing time: 0.8980011940002441
Image scene00506.jpeg of 2700 processing time: 0.9030003547668457
Image scene00507.jpeg of 2700 processing time: 0.8250012397766113
Image scene00508.jpeg of 2700 processing time: 0.8030281066894531
Image scene00509.jpeg of 2700 processing time: 0.8240005970001221
Image scene00510.jpeg of 2700 processing time: 0.8840012550354004
Image scene00511.jpeg of 2700 processing time: 0.868999719619751
Image scene00512.jpeg of 2700 processing time: 0.9009964466094971
Image scene00513.jpeg of 2700 processing time: 0.8220000267028809
Image scene00514.jpeg of 2700 processing time: 0.7920007705688477
Image scene00515.jpeg of 2700 processing time: 0.8189997673034668
Image scene

...

Image scene02126.jpeg of 2700 processing time: 0.9789671897888184
Image scene02127.jpeg of 2700 processing time: 0.9759995937347412
Image scene02128.jpeg of 2700 processing time: 0.9690463542938232
Image scene02129.jpeg of 2700 processing time: 0.9889869689941406
Image scene02130.jpeg of 2700 processing time: 0.9849679470062256
Image scene02131.jpeg of 2700 processing time: 1.0239989757537842
Image scene02132.jpeg of 2700 processing time: 1.0080013275146484
Image scene02133.jpeg of 2700 processing time: 0.9779982566833496
Image scene02134.jpeg of 2700 processing time: 0.9800002574920654
Image scene02135.jpeg of 2700 processing time: 0.9850296974182129
Image scene02136.jpeg of 2700 processing time: 0.9960048198699951
Image scene02137.jpeg of 2700 processing time: 0.9639959335327148
Image scene02138.jpeg of 2700 processing time: 0.989966869354248
Image scene02139.jpeg of 2700 processing time: 0.9720304012298584
Image scene02140.jpeg of 2700 processing time: 0.9619698524475098
Image scene

Image scene02251.jpeg of 2700 processing time: 1.0000033378601074
Image scene02252.jpeg of 2700 processing time: 1.0479984283447266
Image scene02253.jpeg of 2700 processing time: 1.024000883102417
Image scene02254.jpeg of 2700 processing time: 1.0469982624053955
Image scene02255.jpeg of 2700 processing time: 1.0049982070922852
Image scene02256.jpeg of 2700 processing time: 1.0241937637329102
Image scene02257.jpeg of 2700 processing time: 0.9955863952636719
Image scene02258.jpeg of 2700 processing time: 1.0019264221191406
Image scene02259.jpeg of 2700 processing time: 0.9889988899230957
Image scene02260.jpeg of 2700 processing time: 0.992652177810669
Image scene02261.jpeg of 2700 processing time: 1.022498369216919
Image scene02262.jpeg of 2700 processing time: 0.9799723625183105
Image scene02263.jpeg of 2700 processing time: 1.0009958744049072
Image scene02264.jpeg of 2700 processing time: 0.9820003509521484
Image scene02265.jpeg of 2700 processing time: 0.9930281639099121
Image scene02

Image scene02376.jpeg of 2700 processing time: 1.0210294723510742
Image scene02377.jpeg of 2700 processing time: 1.001967191696167
Image scene02378.jpeg of 2700 processing time: 1.0160026550292969
Image scene02379.jpeg of 2700 processing time: 0.9990003108978271
Image scene02380.jpeg of 2700 processing time: 1.0300006866455078
Image scene02381.jpeg of 2700 processing time: 1.1700012683868408
Image scene02382.jpeg of 2700 processing time: 1.2140004634857178
Image scene02383.jpeg of 2700 processing time: 1.0519993305206299
Image scene02384.jpeg of 2700 processing time: 1.0009987354278564
Image scene02385.jpeg of 2700 processing time: 1.0330321788787842
Image scene02386.jpeg of 2700 processing time: 1.0029699802398682
Image scene02387.jpeg of 2700 processing time: 1.03299880027771
Image scene02388.jpeg of 2700 processing time: 1.0009973049163818
Image scene02389.jpeg of 2700 processing time: 1.0620019435882568
Image scene02390.jpeg of 2700 processing time: 1.0579955577850342
Image scene02

Image scene02501.jpeg of 2700 processing time: 1.0230002403259277
Image scene02502.jpeg of 2700 processing time: 1.0419995784759521
Image scene02503.jpeg of 2700 processing time: 1.319000244140625
Image scene02504.jpeg of 2700 processing time: 1.0419995784759521
Image scene02505.jpeg of 2700 processing time: 1.1199986934661865
Image scene02506.jpeg of 2700 processing time: 1.0379984378814697
Image scene02507.jpeg of 2700 processing time: 1.033001184463501
Image scene02508.jpeg of 2700 processing time: 1.0470027923583984
Image scene02509.jpeg of 2700 processing time: 1.0230002403259277
Image scene02510.jpeg of 2700 processing time: 1.0310273170471191
Image scene02511.jpeg of 2700 processing time: 1.0189690589904785
Image scene02512.jpeg of 2700 processing time: 1.0819981098175049
Image scene02513.jpeg of 2700 processing time: 1.0640015602111816
Image scene02514.jpeg of 2700 processing time: 1.02699875831604
Image scene02515.jpeg of 2700 processing time: 1.0650005340576172
Image scene025

Image scene02626.jpeg of 2700 processing time: 1.093998908996582
Image scene02627.jpeg of 2700 processing time: 1.0380017757415771
Image scene02628.jpeg of 2700 processing time: 1.0520317554473877
Image scene02629.jpeg of 2700 processing time: 1.038970947265625
Image scene02630.jpeg of 2700 processing time: 1.050999402999878
Image scene02631.jpeg of 2700 processing time: 1.077000379562378
Image scene02632.jpeg of 2700 processing time: 1.0530011653900146
Image scene02633.jpeg of 2700 processing time: 1.069000005722046
Image scene02634.jpeg of 2700 processing time: 1.0780305862426758
Image scene02635.jpeg of 2700 processing time: 1.0469677448272705
Image scene02636.jpeg of 2700 processing time: 1.0859990119934082
Image scene02637.jpeg of 2700 processing time: 1.0410003662109375
Image scene02638.jpeg of 2700 processing time: 1.0510327816009521
Image scene02639.jpeg of 2700 processing time: 1.0629699230194092
Image scene02640.jpeg of 2700 processing time: 1.0779993534088135
Image scene0264